In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import optuna
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer, FunctionTransformer, PolynomialFeatures, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression

from torch import nn
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.optim as optim
# !pip install -q pytorch-optimizer
import pytorch_optimizer as optim1


from torchmetrics.functional import mean_squared_error, r2_score
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import CSVLogger
import pytorch_lightning as pl
from torchmetrics.regression import MeanAbsoluteError, MeanSquaredError, R2Score

In [138]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
parkinsons_telemonitoring = fetch_ucirepo(id=189) 
  
# data (as pandas dataframes) 
X = parkinsons_telemonitoring.data.features 
y = parkinsons_telemonitoring.data.targets 
  
# metadata 
print(parkinsons_telemonitoring.metadata) 
  
# variable information 
print(parkinsons_telemonitoring.variables) 


{'uci_id': 189, 'name': 'Parkinsons Telemonitoring', 'repository_url': 'https://archive.ics.uci.edu/dataset/189/parkinsons+telemonitoring', 'data_url': 'https://archive.ics.uci.edu/static/public/189/data.csv', 'abstract': "Oxford Parkinson's Disease Telemonitoring Dataset", 'area': 'Health and Medicine', 'tasks': ['Regression'], 'characteristics': ['Tabular'], 'num_instances': 5875, 'num_features': 19, 'feature_types': ['Integer', 'Real'], 'demographics': ['Age', 'Sex'], 'target_col': ['motor_UPDRS', 'total_UPDRS'], 'index_col': ['subject#'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5ZS3N', 'creators': ['Athanasios Tsanas', 'Max Little'], 'intro_paper': {'ID': 229, 'type': 'NATIVE', 'title': "Accurate Telemonitoring of Parkinson's Disease Progression by Noninvasive Speech Tests", 'authors': 'A. Tsanas, Max A. Little, P. McSharry, L. Ramig', 'venue': 'IEEE Transactions on Bio

In [139]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((4700, 19), (1175, 19), (4700, 2), (1175, 2))

In [140]:
y_train.iloc[:, 1]

3268    18.688
4619    31.685
5681    43.579
3349    49.419
4035    33.423
         ...  
3772    35.223
5191    28.150
5226    23.513
5390    43.563
860     45.956
Name: total_UPDRS, Length: 4700, dtype: float64

In [141]:
cat = ['sex']
num = [i for i in list(X_train.columns) if i not in cat]
cat, num

(['sex'],
 ['age',
  'test_time',
  'Jitter(%)',
  'Jitter(Abs)',
  'Jitter:RAP',
  'Jitter:PPQ5',
  'Jitter:DDP',
  'Shimmer',
  'Shimmer(dB)',
  'Shimmer:APQ3',
  'Shimmer:APQ5',
  'Shimmer:APQ11',
  'Shimmer:DDA',
  'NHR',
  'HNR',
  'RPDE',
  'DFA',
  'PPE'])

In [142]:
# num_pipe = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('power', PowerTransformer(method='yeo-johnson')),
#     ('scaler', StandardScaler()),
#     ('poly', PolynomialFeatures(degree=2, include_bias=False))
# ])
# cat_pipe = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# preprocessor = ColumnTransformer(transformers=[
#     ('num', num_pipe, num),
#     ('cat', cat_pipe, cat)
# ])

In [143]:
# X_train = preprocessor.fit_transform(X_train)
# X_test = preprocessor.transform(X_test)
# pd.DataFrame(X_train)

In [ ]:
def create_pipeline(k_best=50, threshold=0.01, degree=2):
    def log_transform(X):
        return np.log1p(X)
    def sqrt(X):
        return np.sqrt(X)
    def inverse(X):
        return 1 / (X + 1e-6)
    
    num_pipeline = Pipeline(steps=[
        # ('imputer', SimpleImputer(strategy='median')),
        ('power', PowerTransformer(method='yeo-johnson')),
        ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
        ('log', FunctionTransformer(log_transform, validate=True)), 
        ('sqrt', FunctionTransformer(sqrt, validate=True)),
        ('inverse', FunctionTransformer(inverse, validate=True)),
        ('scaler', StandardScaler()),
        ('var', VarianceThreshold(threshold=threshold)),
        ('select', SelectKBest(score_func=f_regression, k=k_best))
    ])

    cat_pipe = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    return ColumnTransformer(transformers=[
        ('num', num_pipeline, num),
        ('cat', cat_pipe, cat)
    ])


In [145]:
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    # 1. UBAH DI SINI: Tambahkan parameter untuk menerima data
    def __init__(self, features, labels):
        # 2. Simpan data yang diterima sebagai atribut class
        self.features = features
        self.labels = labels

    # 3. Metode __len__ harus mengembalikan jumlah total sampel
    def __len__(self):
        return len(self.features)

    # 4. Metode __getitem__ harus mengambil satu sampel berdasarkan indeks
    def __getitem__(self, idx):
        # Ambil fitur dan label pada indeks 'idx'
        x = self.features[idx]
        y = self.labels[idx]
        
        # Pastikan outputnya sudah dalam bentuk tensor (jika belum)
        # Jika X_train dan y_train sudah tensor, baris di bawah tidak wajib
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)
        if not isinstance(y, torch.Tensor):
            y = torch.tensor(y, dtype=torch.float32)
            
        return x, y

# NN

In [146]:
class NNModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NNModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.Mish(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),

            nn.Linear(128, 128),
            nn.Mish(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),

            nn.Linear(128, 128),
            nn.Mish(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),

            nn.Linear(128, 32),
            nn.Mish(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.1),

            nn.Linear(32, 16),
            nn.Mish(),
            nn.BatchNorm1d(16),
            nn.Dropout(0.1),

            nn.Linear(16, output_dim)
        )
    def forward(self, x):
        return self.model(x)
    
class PLNN(LightningModule):
    def __init__(self, input_size, num_classes, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters() 
        self.model = NNModel(input_dim=input_size, output_dim=num_classes)
        self.criterion = nn.MSELoss()
        self.mse = MeanSquaredError(squared=False, num_outputs=num_classes)
        self.r2 = R2Score(multioutput='raw_values')
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs) 
        loss = self.criterion(outputs, labels)
        mse = self.mse(outputs, labels)
        r2 = self.r2(outputs, labels)

        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        self.log('train_rmse', mse.mean(), on_epoch=True, prog_bar=True)
        self.log('train_r2', r2.mean(), on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        mse = self.mse(outputs, labels)
        r2 = self.r2(outputs, labels)

        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_rmse', mse.mean(), on_epoch=True, prog_bar=True)
        self.log('val_r2', r2.mean(), on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optimizer = optim1.AdamW(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer

    def compute_metrics(self, outputs, labels, prefix="val", num_classes=2):
        """
        Hitung metric per loader dengan torchmetrics.functional.
        """
        # Pastikan tensor sudah di-device yang sama
        loss = self.criterion(outputs, labels)

        # r2_score & mse dari torchmetrics.functional
        r2_per_label = r2_score(outputs, labels, multioutput="raw_values")
        mse_per_label = mean_squared_error(outputs, labels, num_outputs=num_classes)
        rmse_per_label = torch.sqrt(mse_per_label)

        return {
            f"{prefix}_loss": loss.item(),
            f"{prefix}_rmse_label_1": rmse_per_label[0].item(),
            f"{prefix}_rmse_label_2": rmse_per_label[1].item(),
            f"{prefix}_rmse_avg": rmse_per_label.mean().item(),
            f"{prefix}_r2_label_1": r2_per_label[0].item(),
            f"{prefix}_r2_label_2": r2_per_label[1].item(),
            f"{prefix}_r2_avg": r2_per_label.mean().item()
        }

    def evaluate_loader(self, loader, prefix="val"):
        """
        Evaluasi seluruh data dalam satu DataLoader.
        """
        self.eval()
        all_outputs, all_labels = [], []
        with torch.no_grad():
            for batch in loader:
                inputs, labels, *_ = batch
                outputs = self(inputs)
                all_outputs.append(outputs)
                all_labels.append(labels)

        all_outputs = torch.cat(all_outputs, dim=0)
        all_labels = torch.cat(all_labels, dim=0)
        return self.compute_metrics(all_outputs, all_labels, prefix)

In [147]:
def objective(trial):
#     global X_train, y_train, X_val, y_val
    
    k_best = trial.suggest_int("k_best", 10, 150)
    threshold = trial.suggest_float("variance_threshold", 0.0, 0.02)
    degree = trial.suggest_int("poly_degree", 1, 2)
    bs = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    
    pipeline = create_pipeline(k_best=k_best, threshold=threshold, degree=degree)
    X_train_pipe = pipeline.fit_transform(X_train, y_train.iloc[:, 1])
    X_val_pipe = pipeline.transform(X_val)

    # Konversi ke tensor
    X_train_tensor = torch.tensor(X_train_pipe, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)  # jika DataFrame
    X_val_tensor = torch.tensor(X_val_pipe, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
    
    # Buat dataloader
    train_ds = MyDataset(X_train_tensor, y_train_tensor)
    val_ds = MyDataset(X_val_tensor, y_val_tensor)
    train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=bs, shuffle=False)
    
    # Buat model Lightning
    model = PLNN(input_size=X_train_tensor.shape[1], num_classes=y_train_tensor.shape[1], learning_rate=1e-3)
    
    trainer = pl.Trainer(
        max_epochs=10,  
        enable_progress_bar=False,
        logger=False,
        enable_checkpointing=False
    )
    
    trainer.fit(model, train_loader, val_loader)
    
    return trainer.callback_metrics["val_mse"].item()


In [148]:
# study = optuna.create_study(direction="minimize")  # minimize RMSE
# study.optimize(objective, n_trials=30, show_progress_bar=True, n_jobs=4)
# print("Best pipeline params:", study.best_params)

Best pipeline params: {'k_best': 78, 'variance_threshold': 0.017951044948856656, 'poly_degree': 1, 'batch_size': 32}

In [ ]:
best_params = {'k_best': 78, 'variance_threshold': 0.017951044948856656, 'poly_degree': 1, 'batch_size': 64}

In [150]:
pipeline = create_pipeline(k_best=best_params['k_best'], 
                           threshold=best_params['variance_threshold'],
                           degree=best_params['poly_degree'])

In [151]:
X_train = pipeline.fit_transform(X_train, y_train.iloc[:, 1])
X_val = pipeline.transform(X_val)

In [152]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((4700, 80), (1175, 80), (4700, 2), (1175, 2))

# dataset and dataloader

In [153]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)

# dataset and dataloader
train_set = MyDataset(X_train, y_train)
train_loader = DataLoader(train_set, batch_size=best_params['batch_size'], shuffle=True)
val_set = MyDataset(X_val, y_val)
val_loader = DataLoader(val_set, batch_size=best_params['batch_size'])

feature, target = next(iter(train_loader))
feature.shape, target.shape

(torch.Size([64, 80]), torch.Size([64, 2]))

In [ ]:
if torch.cuda.is_available():
    accelerator_type = 'gpu'
    devices_to_use = 1
else:
    accelerator_type = 'cpu'
    devices_to_use = 'auto'

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='parkinson-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min'
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    mode='min',
)
lr_monitor_callback = LearningRateMonitor(logging_interval='epoch')

trainer1 = pl.Trainer(
    max_epochs=500,
    callbacks=[checkpoint_callback, early_stopping, lr_monitor_callback],
    logger=CSVLogger("tb_logs", name="simple_model_experiment"),
    accelerator=accelerator_type,
    devices=devices_to_use,
    log_every_n_steps=10,
    deterministic=True
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [155]:
model = PLNN(input_size=X_train.shape[1], num_classes=2, learning_rate=1e-3)

In [156]:
trainer1.fit(model, train_loader, val_loader)


  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | NNModel          | 48.9 K | train
1 | criterion | MSELoss          | 0      | train
2 | mse       | MeanSquaredError | 0      | train
3 | r2        | R2Score          | 0      | train
-------------------------------------------------------
48.9 K    Trainable params
0         Non-trainable params
48.9 K    Total params
0.196     Total estimated model params size (MB)
26        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=300` reached.


In [157]:
trainer1.validate(model, val_loader, ckpt_path='best')

Restoring states from the checkpoint path at C:\Users\dsapu\Project\Python\repo\dlp\parkinson telemonitoring\checkpoints\parkinson-epoch=297-val_loss=6.29.ckpt
Loaded model weights from the checkpoint at C:\Users\dsapu\Project\Python\repo\dlp\parkinson telemonitoring\checkpoints\parkinson-epoch=297-val_loss=6.29.ckpt


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │     6.291067123413086     │
│          val_r2           │    0.9260430335998535     │
│         val_rmse          │    2.4069013595581055     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 6.291067123413086,
  'val_rmse': 2.4069013595581055,
  'val_r2': 0.9260430335998535}]

In [159]:
loaders = {
    "train": train_loader,
    "val": val_loader,
}

# Evaluasi manual kapan saja:
metrics = model.evaluate_loader(loaders["val"], prefix="val")
metrics

{'val_loss': 6.291065692901611,
 'val_rmse_label_1': 2.2460408210754395,
 'val_rmse_label_2': 2.745438575744629,
 'val_rmse_avg': 2.495739698410034,
 'val_r2_label_1': 0.9209657311439514,
 'val_r2_label_2': 0.9319806098937988,
 'val_r2_avg': 0.9264731407165527}